In this project I was working with the zoo dataset https://www.kaggle.com/uciml/zoo-animal-classification?select=zoo.csv .
This dataset provides information about different features of animals and classifies them on different classes.
The objectives was to analyze it and apply different techniques of dimensionality reduction to see the results 

In [ ]:
# Import libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load the dataset and see what it contains
df = pd.read_csv('zoo.csv')
df

#2. Statistics

In [ ]:
# Drop 'animal_name' as it usseles for our task
df.drop(['animal_name'], axis=1, inplace=True)
df.head()

In [ ]:
# Check statistical data 
df.describe()

#3. Check missing values

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

#4. Variance

In [ ]:
df.var()

#5. Correlation. Which are the most correlated features with the target ? That's what I'm going to check now

In [8]:
# Separate the target from the features, save features in 'X' and target in 'y'
X = df.drop('class_type', axis=1)
y = df['class_type']

In [ ]:
# Now we are going to calculate the correlation using Pearson's method.
cor = df.corr()
cor

In [ ]:
# Once we got the correlation table save the absolute values in 'cor_target' variable and check which of them are above 0.5 
cor_target = abs(cor["class_type"])

# Our threshold for this task is 0.5 so we will say that every feature that is above 0.5 of correlation is highly correlated with the target.
relevant_features = cor_target[cor_target>0.5]
relevant_features

#6. Which feature has the maximum correlation value?

In [ ]:
max_cor = relevant_features.drop('class_type').idxmax()
max_cor

After applying Pearson's method we can see that the most correlated feature is 'backbone' with a value of 0.828845.

#7. Now let's apply Spearman's method to check if there is any difference with the method we applied previously

In [ ]:
# Apply Spearman's method
spearman_corr = df.corr(method='spearman')
spearman_corr

In [ ]:
# We can make a heatmap to see every correlation value between the features but I'm not doing that in this project.

# Let's use the same code than before to check which features values are above 0.5
spearman_cor_target = abs(spearman_corr['class_type'])

features = spearman_cor_target[spearman_cor_target > 0.5]
features

In [ ]:
# Finally we can see that now the feature with the max correlation value is 'milk' with a value of 0.886024
max_cor_spearman = features.drop('class_type').idxmax()
max_cor_spearman

Using both methods, the correlations change.

Using the Pearson method, the variable with the highest correlation is 'backbone,' while using Spearman, it is 'milk.' Additionally, the correlations of the other variables differ from one method to another.

This is because the Pearson method measures the linear correlation between variables, that is, how well they align with a direct linear relationship. This method is sensitive to the scale of values and the presence of outliers.

On the other hand, the Spearman method measures the correlation of variables with ordinal data, assessing whether a relationship exists even when the data is non-linear. It uses the rank of the values instead of the values themselves, making it less sensitive to outliers and non-linear relationships.

#8. Now let's work with backward elimination. Which features do we keep?

In [ ]:
# To apply backward elimination we are going to use the OLS algorithm (Ordinary Least Squares), this method uses the p_value to identify which columns has to keep to train the model

import statsmodels.api as sm

cols = list(X.columns)
X_1 = X[cols]
X_1 = sm.add_constant(X_1)
model = sm.OLS(y,X_1).fit()
model.summary()
# When 

In [ ]:
# Now there's the implementation of the backaward algorithm
import statsmodels.api as sm


cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p = []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)


""" As we can see in the output now we have 7 features left of the 17 we had at the beginning
['feathers', 'milk', 'airborne', 'aquatic', 'toothed', 'backbone', 'fins'] """

#9. Which is the optimum number of features when applying RFE? Which are them ?

In [17]:
# The RFE algorithm classifies the columns from most to least important instead of using the p_value

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

In [18]:
# Let's create a Lienar Regression model and select 8 features
model = LinearRegression()
rfe = RFE(model, n_features_to_select=8)

In [ ]:
X_rfe = rfe.fit(X,y)
X_rfe = rfe.transform(X)
model.fit(X_rfe,y)

In [ ]:
# After training the columns that appear with a 1 are the most important ones

print(rfe.support_)
print(rfe.ranking_)

In [ ]:
# Now lets check the optimal number and the score when using those columns
from sklearn.model_selection import train_test_split

#no of features
nof_list=np.arange(1,17)
high_score=0
#Variable to store the optimum features
nof=0
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model, n_features_to_select=nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

In [22]:
# Last let's check the name of the columns we worked with
cols = list(X.columns)
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, n_features_to_select=3)
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)
#Fitting the data to model
model.fit(X_rfe,y)
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index

In [ ]:
# We check the name of the most optimal columns
selected_features_rfe

#10. Now lets work with RIDGE and LASSO, let's check how many features each of the algorithm drops.

In [ ]:
# Firstly let's check which variables RIDGE drops, this algorithm penalize the components to not overfit and makes the feature selection during training

from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
import matplotlib.pyplot as plt

reg_ridge = RidgeCV()
reg_ridge.fit(X, y)
coef = pd.Series(reg_ridge.coef_, index=X.columns)

features_chosen = coef[coef != 0].index.tolist()
features_dropped = coef[coef == 0].index.tolist()

print("Ridge chose " + str(sum(coef != 0)) + " features and dropped " +  str(sum(coef == 0)) + " features")
print('Features chosen', features_chosen)
print('Features dropped', features_dropped)

In [ ]:
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Importance of variables using the Ridge Model")

In [ ]:
reg_lasso = LassoCV()
reg_lasso.fit(X, y)
coef_2 = pd.Series(reg_lasso.coef_, index=X.columns)

features_chosen_2 = coef_2[coef_2 != 0].index.tolist()
features_dropped_2 = coef_2[coef_2 == 0].index.tolist()

print("Lasso chose " + str(sum(coef_2 != 0)) + " features and dropped " +  str(sum(coef_2 == 0)) + " features")
print('Features chosen', features_chosen_2)
print('Features dropped', features_dropped_2)

In [ ]:
imp_coef_2 = coef_2.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef_2.plot(kind = "barh")
plt.title("Importance of variables using the Lasso Model")

As we can see RIDGE doesn't drop any feature while LASSO drops 2 which are 'venomous' and 'domestic'

#11. Using 2 components which weight has the 'domestic' feautre in component 1?

In [ ]:
# Let's use PCA to check that

# First we have to check the variance, if there's one that is very high we have to do something
X.var()

In [ ]:
# As 'legs' has a high value over the others let's use StandardScaler on all of our data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(X_scaled)

# As we only want to see the weight we don't need to transform , just fitting we are going to be able to see what we need
cargas = pd.DataFrame(pca.components_, columns=X.columns, index=['PC1', 'PC2'])
print(f'Weight of the features in the principal components:\n{cargas}')

In [ ]:
# Now let's filter to see what we exactly need to see
peso_domestic_pc1 = cargas.loc['PC1', 'domestic']
print(f'El peso de la variable domestic en la componente 1 es: {peso_domestic_pc1}')

In [ ]:
# If we want to plot everything we can apply the following code (Change the title to your language, in this case the titles and labels are in Spanish)
plt.figure(figsize=(10, 6))
cargas.T.plot(kind='bar', figsize=(10, 6), width=0.8) # Se hace para transponer el dataframe y que las variables estén en el eje X y las componentes en el eje Y
plt.title("Cargas de las variables en las componentes principales")
plt.xlabel("Variables")
plt.ylabel("Cargas")
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8) # Añade una línea en Y=0 para diferenciar entre las cargas positivas y negativas
plt.legend(title="Componentes principales")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# We can also use a heatmap to visualize it
import seaborn as sns

plt.figure(figsize=(12, 7))
sns.heatmap(cargas, annot=True, cmap="coolwarm", center=0, cbar_kws={'label': 'Cargas'})
plt.title("Cargas de las variables en las componentes principales")
plt.xlabel("Variables")
plt.ylabel("Componentes principales")
plt.show()

In [ ]:
# To only plot the 'domestic' values we can do it like this
pesos_domestic = cargas['domestic']

# Graficar
plt.figure(figsize=(6, 4))
pesos_domestic.plot(kind='bar', color=['blue', 'orange'], alpha=0.8)
plt.title("Peso de la variable 'domestic' en las componentes principales")
plt.ylabel("Peso (carga)")
plt.xlabel("Componentes principales")
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

#12. How many coeficients do we obtain in LDA ?


In [ ]:
# LDA is a supervised learning algorithm used for classification, but it is also utilized in dimensionality reduction tasks as it seeks the combination of features that best separates the classes 
# in a dataset.
# To ensure this algorithm works correctly, we need a Gaussian distribution and similar covariance matrices.
# In our case, if we observe X, it does not have a Gaussian distribution, but if we observe X_scaled, it does.

sns.histplot(X_scaled)

In [ ]:
# Apply LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda.fit(X_scaled, y)

In [ ]:
# Get the coeficients
coeficientes = pd.DataFrame(lda.coef_, columns=X.columns)
print('Número de coeficientes:\n', coeficientes)

We can observe that we obtain 6 coefficients for each variable. This is due to how LDA calculates the coefficients, using the formula min(k-1, p), where k is the number of classes and p is the total number of variables. Thus, when we substitute the values (7-1 = 6), we get min(6, 16) = 6, resulting in 6 coefficients per variable.

The absolute values of the coefficients indicate the importance of each variable for each discriminant. Therefore, the variables with the highest absolute values in each discriminant signify that those variables have a more significant impact on the discrimination between classes for that discriminant.